# Introducción 

La idea de la práctica es visitar aquellos temas que en cierta manera nos permitan ver más contenido del curso.

La práctica esta dividida en 4 o 5 subapartados, que ya tenéis en este mismo Notebook. Estos subapartados estan aquí para que rellenéis el código que hace falta para la realización de la práctica. Obviamente podéis usar tantas celdas como os hagan falta, es más es de agradecer si el código final esta algo "limpio". Usar funciones, algo de comentario, etc, etc...

Usaremos 2 datasets, uno para el primer ejercicio, y otro para el resto de ejercicios.

Ejercicios:


1.   Machine Learning vs Deep Learning (Acordaros que hay que implementar el pipeline visto en clase entero)

    1.1. Implementación de un modelo de Sentiment Analysis con algún algoritmo de Machine Learning Clásico.
    
    1.2. Implementación de un modelo de Sentiment Analysis con alguna arquitectura de Deep Learning.
    
    1.3. Breve Comparación de resultados. Confusion Matrix.
    
2. Hacer Analysis de los tweets del segundo dataset. Que temas aparecen? Como se representan estos temas? De que hablan unos y otros?

3. Escoged a uno de los dos presidentes, y escribid tweets como ellos, usando un Modelo Generativo.

En cada ejercicio, espero explicaciones y razonamientos del porque una arquitectura y no otra, por ejemplo en Deep Learning, porque usar Convolutionals en lugar de recurrentes, o en Machine Learning, Bayes en lugar de SVM. Hay que explicar el pipeline, sobretodo el preproceso de datos, con lo que habrá que hacer un pequeño estudio de que datos tenemos, y si hay cosas que se pueden ignorar, si hacéis stemming, o no, etc, etc...

Acordaros de que objetivo final no es que obtengáis una accuracy brutal, es que comprendais que pasa cuando usais un algoritmo u otro, y que problemas o beneficios nos dan.

![](https://i.pinimg.com/736x/19/63/8c/19638c0b33e2f7822d6806ce31d89d84--funny-cartoons-funny-jokes.jpg =400x)

Mucha suerte y ánimo!



In [0]:
!pip install spacy
!python -m spacy download en_core_web_md

In [38]:
import spacy
import numpy as np
import pickle
import json
import os
import csv
import pprint as pp
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from random import shuffle, choice, sample
from sklearn.model_selection import StratifiedShuffleSplit
from copy import copy
import warnings
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
from IPython import display
from keras.models import Model, Sequential
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Merge, Dot, Concatenate, Flatten, Permute, Multiply, dot, concatenate
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.models import load_model
from google.colab import files
from collections import Counter

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
warnings.filterwarnings('ignore')
sns.set(color_codes=True)
nlp = spacy.load('en_core_web_md')

In [11]:
uploaded = files.upload()

Saving dataset2.json to dataset2 (1).json


Comenzamos cargando los dato del json en un dataframe de pandas para analizarlos

In [14]:
tweets = json.loads(uploaded['dataset2.json'])
df =  pd.DataFrame([[x['tweet'], x['label']] for x in tweets.values() ], columns=['tweet','label'])
df.head()

,tweet,label
0,"“Low plastic stool, cheap but delicious noodle...",OBAMA
1,"“Low plastic stool, cheap but delicious noodle...",OBAMA
2,"This National Gun Violence Awareness Day, show...",OBAMA
3,We can never truly repay the debt we owe our f...,OBAMA
4,This Center is for the leaders of tomorrow who...,OBAMA


Tenemos textos de tweets de en la primera columna y quein los ha escrito en la segunda. En este caso vamos a utilizar solo los de TRUMP

###Preparamos el dataset

In [0]:

tweetsTRUMP = df[df['label'] == 'TRUMP']['tweet'].tolist()
tokenized = [list(x) for x in tweetsTRUMP]  #Caracteres

In [0]:

#Caracteres con los que puede empezar la frase, sirve de inicio al algoritmo para predecir
init_chars = [x[:5] for x in tokenized]
for i in range(len(init_chars)):
    tmp = init_chars[i]
    tmp.insert(0, '<SOS>')
    init_chars[i] = tmp[:5]   

In [37]:

#ongitus máxima y media de las frases
maxlen = max([len(x) for x in tokenized])
avglen = sum([len(x) for x in tokenized])/len(tokenized)
print(maxlen, avglen)

320 160.39098497495826


In [67]:
#Caracteres que forman el vocabulario
vocab = [t for s in tweetsTRUMP for t in s]
print('num tokens: {}'.format(len(vocab)))
vocab_counter = Counter(vocab)
vocab = [w for w, v in vocab_counter.items() if v>2]
vocab = ['<PAD>', '<UNK>', '<SOS>', '<EOS>'] + vocab
nb_vocab = len(vocab)
print('num features a user {}'.format(nb_vocab))
w2id = {k:i for i, k in enumerate(vocab)}
id2w = {i:k for k, i in w2id.items()}

num tokens: 480371
num features a user 111


In [43]:
maxlen = min(maxlen, 5)

step = 1
data_train = []
for x in tokenized:
    x.insert(0, '<SOS>')
    x.append('<EOS>')
    for i in range(0, len(x)-maxlen, step):
        data_train.append((x[i:i+maxlen], x[i+maxlen]))
        
print('nb_sequences: {}'.format(len(data_train)))

nb_sequences: 471386


## Definición del modelo

Vamos a utilizar el modelo visto en clase, que utilizará embedings y una lstm bidireccional

Los datos los hemos preparado para que sean secuencias de caracteres, por lo que la red aprendera secuencias de caracteres en el tiempo, por lo que en el momento de predecir, le tendremos que dar una secuencia de caracteres inicial y el modelo a partir de ahi generará el resto de la frase.



In [0]:
SAMPLE_EVERY = 5

In [0]:
def sample_pred(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [0]:
class Sampletest(Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % SAMPLE_EVERY == 0  and epoch>0:
            data_test = []
            nb_samples = 1
            
            params = {
                'maxlen': maxlen,
                'vocab': nb_vocab,
                'use_embeddings': True
                }
            for _ in range(nb_samples):
                data_test = choice(init_chars)
                for diversity in [0.2, 0.6, 1.2]:
                    print('----- diversity:', diversity)
                    sentence = copy(data_test)
                    generated = copy(data_test)
                    for i in range(len(data_test), 400):
                        x_pred = np.zeros((1, params['maxlen']))
                        for t, char in enumerate(sentence):
                            x_pred[0, t] = w2id[char] if char in w2id else w2id['<UNK>']
                        preds = self.model.predict(x_pred, verbose=0)[0]
                        next_index = sample_pred(preds, diversity)
                        next_char = id2w[next_index]
                        if next_char == '<EOS>':
                            break
                        generated += [next_char]
                        sentence = sentence[1:] 
                        sentence += [next_char]
                    print(''.join(generated))


In [0]:
class HistoryDisplay(Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % PLOT_EVERY == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()
        

In [0]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [0]:
class LM:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
    
    def compile_bidirectional(self, params={}):
        
        lm_inputs = Input(shape=(params['maxlen'], ), name='lm_input')
        
        embeddings = Embedding(params['vocab'], params['emb_feats'])(lm_inputs)
        
        lstm =  CuDNNLSTM(params['rnn_hidden'], return_sequences=True, name='rnn1')        
        
        lmlstm = Bidirectional(lstm)(embeddings)       
        
        stacklstm =  CuDNNLSTM(params['rnn_hidden'], return_sequences=False, name='stacked')
        
        stackedlstm = stacklstm(lmlstm)
        
        lmout = Dense(params['vocab'], activation='softmax')(stackedlstm)
        
        model = Model(lm_inputs, lmout)
        
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        
        model.summary()
        
        return model
        
    def train(self, model, data, params={}):
        
        callbacks = self._get_callbacks()
        
        if 'shuffle' in params and params['shuffle']:
            shuffle(data)
            
        sentences, next_chars = zip(*data)
        print(sentences[0])
        x = np.zeros((len(data), params['maxlen']))
        y = np.zeros((len(data), params['vocab']))
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t] = w2id[char] if char in w2id else w2id['<UNK>']
            y[i, w2id[next_chars[i]] if next_chars[i] in w2id else w2id['<UNK>']]  = 1
        
        model.fit(x, y, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks, verbose=1)

    def predict(self, model, data, params={}):        
        if 'use_embeddings' in params and params['use_embeddings']:
            # variedad en las predicciones
            for diversity in [0.2, 0.6, 1.2]:
                print('----- diversity:', diversity)
                sentence = copy(data)
                generated = copy(data)
                # cuantas predicciones queremos hacer
                for i in range(len(data), 400):
                    x_pred = np.zeros((1, params['maxlen']))
                    # preparar inpunt
                    for t, char in enumerate(sentence):
                        x_pred[0, t] = w2id[char] if char in w2id else w2id['<UNK>']
                    # predecir
                    preds = model.predict(x_pred, verbose=0)[0]
                    next_index = sample_pred(preds, diversity)
                    next_char = id2w[next_index]
                    # mirar si hemos terminado
                    if next_char == '<EOS>':
                        break
                    # ana                        
                    generated += [next_char]
                    sentence = sentence[1:] 
                    sentence += [next_char]
                print(''.join(generated))
    
    
    def load(self, model_path='seq2seq_attn.h5'):
        return load_model(model_path)
    
    def _get_callbacks(self, model_path='seq2seq_attn.h5'):
        
        
        es = EarlyStopping(monitor='loss', patience=4, mode='auto', verbose=0)       
        
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose = 0, save_best_only=True, save_weights_only=False, period=2)
        st = Sampletest()
        # hd = HistoryDisplay()
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)
        return [st, rlr]

In [0]:
compile_params = {
    'maxlen': maxlen, 
    'vocab': len(vocab),
    'emb_feats': 100,
    'rnn_hidden': 256,
}

In [0]:
LOAD_MODEL = False
bTrain = True

In [80]:
lm = LM()
if LOAD_MODEL:
    path = 'final_{}.h5'.format(dtype)
    lm_model = lm.load(model_path=path)
    lm_model.summary()
else:
    lm_model = lm.compile_bidirectional(params=compile_params)    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lm_input (InputLayer)        (None, 5)                 0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 5, 100)            11100     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 512)            733184    
_________________________________________________________________
stacked (CuDNNLSTM)          (None, 256)               788480    
_________________________________________________________________
dense_2 (Dense)              (None, 111)               28527     
Total params: 1,561,291
Trainable params: 1,561,291
Non-trainable params: 0
_________________________________________________________________


## Entrenamiento del modelo

A continuación pasamos a entrenar el modelo, durante el entrenamiento se van imprimiendo ejemplos de frases generadas de modo que se pueda ir apreciiando como evoluciona el modelo

In [85]:
train_params = {
    'epochs': 50,
    'batch_size': 512,
    'shuffle': True,
    'vocab': nb_vocab,
    'maxlen': maxlen,
    'use_embeddings': True
}
pp.pprint(train_params)
if bTrain:
    lm.train(model=lm_model, data=data_train, params=train_params)

{'batch_size': 512,
 'epochs': 50,
 'maxlen': 5,
 'shuffle': True,
 'use_embeddings': True,
 'vocab': 111}
['s', 't', 'e', 'r', ' ']
Epoch 1/50
443904/471386 [===========================>..] - ETA: 1s - loss: 1.2217 - acc: 0.6531

471386/471386 [==============================] - 26s 55us/step - loss: 1.2231 - acc: 0.6526
Epoch 2/50
471386/471386 [==============================] - 26s 55us/step - loss: 1.1894 - acc: 0.6593
Epoch 3/50
 35328/471386 [=>............................] - ETA: 24s - loss: 1.1425 - acc: 0.6700

471386/471386 [==============================] - 26s 55us/step - loss: 1.1635 - acc: 0.6644
Epoch 4/50
342528/471386 [====================>.........] - ETA: 7s - loss: 1.1358 - acc: 0.6705

471386/471386 [==============================] - 26s 55us/step - loss: 1.1410 - acc: 0.6694
Epoch 5/50
471386/471386 [==============================] - 26s 55us/step - loss: 1.1197 - acc: 0.6740
----- diversity: 0.2
<SOS>.@foxandfriends the United States the problem is a truly all of the United States the world to be really be a great honor to working our country is being so much more the best the United States that the proved the most all of the United States to the terrorist attack in London. She is strong and a fantastic missile that the United State of the terrorist attack in London. She is strong and the pro
----- diversity: 0.6
<SOS>.@foxandfriends. So much more that the United States morning! Unemployment was a great honor to welcome that happen!
----- diversity: 1.2


<SOS>.@foxandfriends. Here. I legislatives, yet the 13 bill Deeplions about Crascorded so class. T
Epoch 6/50
471386/471386 [==============================] - 26s 54us/step - loss: 1.1008 - acc: 0.6775
Epoch 7/50
 32256/471386 [=>............................] - ETA: 24s - loss: 1.0466 - acc: 0.6912

...

471386/471386 [==============================] - 27s 57us/step - loss: 0.8438 - acc: 0.7297
----- diversity: 0.2
<SOS>...and the most in the terrorist attack in London. She is strong and security and doing very well.
----- diversity: 0.6
<SOS>...and the campaign was the terrible with our country is the truly great Again!
----- diversity: 1.2
<SOS>...and low amounts former and will be around. Specially bad products-n Hurricans were has agreement... https://t.co/GuRolan defeat idea Companies ahead In. MaybeAmerican American working close list. Donzon must put crime
Epoch 46/50
 92672/471386 [====>.........................] - ETA: 21s - loss: 0.8158 - acc: 0.7377

471386/471386 [==============================] - 27s 56us/step - loss: 0.8424 - acc: 0.7302
Epoch 47/50
367104/471386 [======================>.......] - ETA: 5s - loss: 0.8351 - acc: 0.7321

471386/471386 [==============================] - 27s 56us/step - loss: 0.8413 - acc: 0.7303
Epoch 48/50
471386/471386 [==============================] - 26s 56us/step - loss: 0.8404 - acc: 0.7298
Epoch 49/50
 10752/471386 [..............................] - ETA: 26s - loss: 0.7940 - acc: 0.7441

471386/471386 [==============================] - 27s 57us/step - loss: 0.8390 - acc: 0.7308
----- diversity: 0.2
<SOS>Spoke to U.K. Prime Minister There is strong and the terrorist attack in London. She is strong and the terrorist attack in London. She is strong and prayers are will be a great honor to have been a very well.
----- diversity: 0.6
<SOS>Spoke to U.K. Prime Minister There is a true friends
----- diversity: 1.2
<SOS>Spoke to built authorizationalAgricultureDay
Epoch 50/50
107008/471386 [=====>........................] - ETA: 20s - loss: 0.8120 - acc: 0.7374

471386/471386 [==============================] - 27s 56us/step - loss: 0.8379 - acc: 0.7306


En el modelo hemos conseguido un accuracy del 73%, aunque por la evolución tiene pinta de que con mas époccas se hubiera conseguido un mejor accuracy

## Generación de frases

Por ultimo generaremos frases de manera aleatoria. Cogeremos unos caracteres iniciales de manera aleatoria y el modelo generara 3 frases, cada una con un valor del parametro diversity, que indica el grado de libertad del modelo.

Las frases con el diversity mas bajo, son mas coherentes y reproducen mas las frases originales. Cuanto mas lo aumentamos se ve que la frase tiene menos sentido pero puede crear frases diferentes a las originales

In [86]:
data_test = choice(init_chars)

                    
lm.predict(model=lm_model, data=data_test, params=train_params)

----- diversity: 0.2
<SOS>I will be a great honor to welcome to U.K. Prime Minister Theresa May to our great honor to be with the U.S. is being me when the Democrats are with the Fake News Media is on the election on the terrorist attack in London. She is strong and the U.S. is workers and the terrorist attack in London. She is strong and the U.S. Coast Guard to be a great honor to welcome to U.K. Prime Minister Theresa M
----- diversity: 0.6
<SOS>I will be the world so than the terrorist attack in London. She is strong and and the Democrats are really basics and Whistleblower to welcome the Fake News story that the men and the Fake News Media has been action and open our military, and doing very well.
----- diversity: 1.2
<SOS>I will control, I have no such giant family. Mi)sid, 33,000 lives them to the larger the Russia, Reciprocity. Hillary or campaign murder Laws are cl🇺3l3JbE6Gx9
